# Using the ModelMW python client to run watershed analysis on HUCs in bulk



## Import libraries

In [1]:
from IPython.display import display

from modelmw_client import *

## Create an API client

In this case, I'm importing my saved credential information from a separate python file, but the credentials can be stored anywhere.

In [2]:
from mmw_secrets import (
    srgd_staging_api_key,
    srgd_mmw_user,
    srgd_mmw_pass,
)

# Create an API user
mmw_run = ModelMyWatershedAPI(api_key=srgd_staging_api_key, use_staging=True)
# Authenticate with MMW, optional
mmw_run.login(mmw_user=srgd_mmw_user, mmw_pass=srgd_mmw_pass)


True

## Set up the areas of interest

To run analysis on a batch of areas of interest, we need a list of those areas.
We can either have a list of HUC codes (8, 10, or 12 digit HUCs) or a list of single-feature geojsons.
The list can be imported from a csv or other file.

If you're using geojsons, I recommend using the [python geojson library](https://pypi.org/project/geojson/) in combination with [GeoPandas](https://geopandas.org/en/stable/) or [SpatialPandas](https://github.com/holoviz/spatialpandas).
All of those packages use [fiona](https://fiona.readthedocs.io/en/latest/manual.html) for reading and writing geometry files.

For this example, we're just going to use a simple list of HUC codes.
These are the HUC 12's within the White Clay Creek HUC 10.

In [3]:
hucs_to_analyze = ['020402050301', '020402050302', '020402050303', '020402050304', '020402050305', '020402050306', '020402050307', '020402050308']

## Run batch analysis of land use

To analyse land use, we need to pick an NLCD layer source.  
The available land use layers are:

- `2019_2019` - Data from the 2019 survey
- `2019_2016` - Data from the 2016 survey, reprocessed using the 2019 methodolgy
- `2019_2011` - Data from the 2011 survey, reprocessed using the 2019 methodolgy
- `2019_2006` - Data from the 2006 survey, reprocessed using the 2019 methodolgy
- `2019_2001` - Data from the 2001 survey, reprocessed using the 2019 methodolgy
- `2011_2011` - Data from the 2011 survey, as processed using the **original 2011** methodology

The corresponding analysis end point for each NLCD layer can be obtained using `mmw_run.land_endpoint.format({layer})`.
In this example, we will use the 2019 NLCD survey.
The analysis takes several seconds for each area of interest.

Under the hood, the ModelMyWatershed python client is making at least two calls to ModelMyWatershed.
First, there is a call to the given analysis endpoint to start the analysis job.
ModelMyWatershed responds with a UUID for the job.
To get the analysis results, the client then calls the job endpoint using the job UUID.
The results may take up to a few seconds to be available; before they are available, the job endpoint will return nothing.

In [4]:
print(mmw_run.land_endpoint.format("2019_2019"))
land_use_frame = mmw_run.run_batch_analysis(
    hucs_to_analyze, mmw_run.land_endpoint.format("2019_2019")
)
display(land_use_frame)


api/analyze/land/2019_2019/
	land_2019_2019 job started for 020402050301
	Got land_2019_2019 results for 020402050301
	land_2019_2019 job started for 020402050302
	Got land_2019_2019 results for 020402050302
	land_2019_2019 job started for 020402050303
	Got land_2019_2019 results for 020402050303
	land_2019_2019 job started for 020402050304
	Got land_2019_2019 results for 020402050304
	land_2019_2019 job started for 020402050305
	Got land_2019_2019 results for 020402050305
	land_2019_2019 job started for 020402050306
	Got land_2019_2019 results for 020402050306
	land_2019_2019 job started for 020402050307
	Got land_2019_2019 results for 020402050307
	land_2019_2019 job started for 020402050308
	Got land_2019_2019 results for 020402050308


,area,active_river_area,code,coverage,nlcd,type,job_label,request_endpoint
0,2.332910e+04,3.589092e+03,open_water,0.000663,11,Open Water,020402050301,api/analyze/land/2019_2019/
1,0.000000e+00,0.000000e+00,perennial_ice,0.000000,12,Perennial Ice/Snow,020402050301,api/analyze/land/2019_2019/
2,7.545169e+06,1.591762e+06,developed_open,0.214335,21,"Developed, Open Space",020402050301,api/analyze/land/2019_2019/
3,1.938110e+06,2.978947e+05,developed_low,0.055056,22,"Developed, Low Intensity",020402050301,api/analyze/land/2019_2019/
4,5.069593e+05,6.280912e+04,developed_med,0.014401,23,"Developed, Medium Intensity",020402050301,api/analyze/land/2019_2019/
...,...,...,...,...,...,...,...,...
123,1.552301e+05,5.742617e+04,grassland,0.002178,71,Grassland/Herbaceous,020402050308,api/analyze/land/2019_2019/
124,2.496244e+06,8.793382e+05,pasture,0.035026,81,Pasture/Hay,020402050308,api/analyze/land/2019_2019/
125,7.707669e+05,3.131521e+05,cultivated_crops,0.010815,82,Cultivated Crops,020402050308,api/analyze/land/2019_2019/
126,3.370198e+06,1.714710e+06,woody_wetlands,0.047289,90,Woody Wetlands,020402050308,api/analyze/land/2019_2019/


## Finished!

We now have land use distributions for all of the areas of interest (our HUCs).
We can look at the table, use it, and save it in any pandas-supported format.

If we want data from more analysis endpoints (such as soil), we can use the `run_batch_analysis(...)` function with whatever endpoint we want.
The possible endpoints are:

- land_endpoint: For land use derived from the NLCD surveys
  - As shown above, you must specify the surve and analysis year for the NLCD endpoint
- streams_endpoint: For stream network statistics; can use either the NHD high resolution stream layer or the NHD medium resolution layer
- protected_lands_endpoint: For protected lands in the Protected Areas Database of the United States (PADUS), published by the U.S. Geological Survey Gap Analysis Program in 2016.
- soil_endpoint: For the Hydrologic Soil Groups From USDA gSSURGO 2016
- terrain_endpoint: For summary statistics of elevation and slope using the NHDPlus V2 NEDSnapshot DEM
- climate_endpoint: For a monthly climate (precipitation and mean temperature) summary from the PRISM Climate Group
- point_source_endpoint: For a discharge monitoring report of annual average inputs for the given area based on the data from the EPA NPDES
- animal_endpoint: For estimated counts of livestock animal density in the given area based on county-level averages from the USDA
- catchment_water_quality_endpoint: For calibrated GWLF-E model estimates derived by the Stream Reach Assessment Tool (SRAT) based on 2011 data within the Delaware River Basin (*not national data*)
- forcast_endpoint: For simulated land use in the year 2100 in the Delaware River Basin (*not national data*)
  - Uses simulations of change in land use over the 20-year period 2080-2099 based on two urban growth scenarios: centers and corridors. 
Generated by Shippensburg University, serviced via APIs by Drexel University and the Academy of Natural Sciences.

Further information on the source data behind each of these endpoints is available in the [ModelMyWatershed technical documents](https://wikiwatershed.org/help/model-help/mmw-tech/).